In [78]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

search = arxiv.Search(
    query = "Attention is all you need",
    max_results = 10,
    sort_by=arxiv.SortCriterion.SubmittedDate, 
    sort_order=arxiv.SortOrder.Descending
)

results = client.results(search)
# `results` is a generator; you can iterate over its elements one by one...
for r in client.results(search):
    print(r.download_pdf())
    print()
    break

./2404.15626v1.An_Electromagnetism_Inspired_Method_for_Estimating_In_Grasp_Torque_from_Visuotactile_Sensors.pdf



In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("assets/arxiv_paper.pdf")
loader.load()

## Инициализация модели
Теперь инициализируем модель GigaChat.

In [46]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

In [47]:
from langchain.chat_models.gigachat import GigaChat
llm = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("assets/arxiv_paper.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

После нарезки мы получили 91 документ частями книги.

## Создание базы данных эмбеддингов

Эмбеддинг это векторное представление текста, которое может быть использовано для определения смысловой близости текстов. Векторная база данных хранит тексты и соответствующие им эмбеддинги, а также умеет выполнять поиск по ним. Для работы с базой данных мы создаем объект GigaChatEmbeddings и передаем его в базу данных Chroma.

> Обратите внимание, что сервис для вычисления эмбеддингов может тарифицироваться отдельно от стоимости модели GigaChat.

In [51]:
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embeddings = GigaChatEmbeddings(
    credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

## Поиск по базе данных

Теперь можно обратиться к базе данных и попросить найти документы, которые с наибольшей вероятностью содержат ответ на наш вопрос.

По-умолчанию база данных возвращает 4 наиболее релевантных документа. Этот параметр можно изменить в зависимости от решаемой задачи и типа документов.

Видно, что первый же документ содержит внутри себя часть книги с ответом на наш вопрос.

In [52]:
docs = db.similarity_search(question, k=4)
len(docs)

4

In [53]:
print(f"... {str(docs[0])[620:800]} ...")

... ение (fork) открытой библиотеки LangСhain на Python. Её главная цель — облегчить жизнь разработчику. Библиотека состоит из большого количества различных компонентов, которые позвол ...


In [ ]:
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate

propmpt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            #             prompt = """Сгенерируй от {dataset_size_min} до {dataset_size_max} синонимов для слова "{subject}".
            # Результат верни в формате JSON списка без каких либо пояснений, например, ["синоним1", "синоним2", "синоним3", "синоним4"].
            # Не дублируй фразы."""
            prompt=load_prompt("lc://prompts/synonyms/synonyms_generation.yaml")
        )
    ]
)

## QnA цепочка

Теперь мы создадим цепочку QnA, которая специально предназначена для ответов на вопросы по документам. В качестве аргументов есть передается языковая модель и ретривер (база данных).

In [54]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [57]:
print(qa_chain)

combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=GigaChat(credentials='YmZlNGMwYWQtM2E0ZS00NzQ3LWIzMzQtZWYxN2NjNTYxODEyOmIyMjkxZTI2LTg4NjktNDc1Yy05NjE5LTg2NzUxNzc3MWZmYg==', scope='GIGACHAT_API_CORP', verify_ssl_certs=False, _client=<gigachat.GigaChat object at 0x7f82e5000940>)), document_variable_name='context') retriever=VectorStoreRetriever(tags=['Chroma', 'GigaChatEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f82e5090a30>)


Наконец можно задать вопрос нашей цепочке и получить правильный ответ!

In [58]:
qa_chain({"query": question})

/workspace/Syrenny/GigaHack/Syrenny/giga_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'Какой Loss использует Yolov8?',
 'result': 'Я не знаю ответа на этот вопрос.'}

Несколько дополнительных вопросов для проверки работоспособности:

In [89]:
qa_chain({"query": "Расскажи про тимлида Сбера"})

{'query': 'Расскажи про тимлида Сбера',
 'result': 'Извините, но у меня нет информации о конкретном тимлиде Сбера. Могу рассказать вам о том, что такое тимлид и какие у него обязанности. Тимлид - это лидер команды, который отвечает за управление командой, ее развитие и достижение поставленных целей. Он координирует работу членов команды, помогает им решать возникающие проблемы, поддерживает мотивацию и вовлеченность. Кроме того, тимлид также участвует в принятии решений, связанных с проектами, и помогает команде в достижении общих целей.'}

In [24]:
qa_chain({"query": "Основные особенности Yolov8"})

{'query': 'Основные особенности Yolov8',
 'result': 'YOLOv8 имеет несколько особенностей, которые отличают его от других версий YOLO. Во-первых, он использует новую архитектуру, которая сочетает в себе модули FAN и PAN. Это позволяет ему лучше захватывать особенности на разных масштабах и разрешениях, что важно для точного обнаружения объектов разного размера и формы. Кроме того, YOLOv8 превосходит YOLOv5 по нескольким параметрам, включая более высокую метрику mAP и меньшее количество выбросов при измерении против RF100. Он также превосходит YOLOv5 для каждого RF100 категории.'}

In [ ]:
key_words = ["NLP", "RAG", "ChatBot", "LLM", "Speech Recognition"]

In [104]:
import arxiv
import os
import json

def download_from_arxiv(key_word, max_results=10, destination_path="assets/arxiv/", saved_list_path="assets/arxiv/papers.json"):
    client = arxiv.Client()

    search = arxiv.Search(
        query = str(key_word),
        max_results = max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate, 
        sort_order=arxiv.SortOrder.Descending
    )
    # Проверка существования файла с сохраненным списком скачанных arXiv ID
    if os.path.exists(saved_list_path):
        with open(saved_list_path, "r") as f:
            downloaded_arxiv_ids = json.load(f)
    else:
        downloaded_arxiv_ids = []
    
    for result in client.results(search):
        # Проверка наличия статьи в списке уже скачанных
        if result.entry_id.split("/")[-1] in downloaded_arxiv_ids:
            print(f"Статья {result.entry_id} уже скачана и пропущена.")
            continue
        
        # Скачивание PDF-файла статьи
        pdf_path = result.download_pdf(dirpath=destination_path)
        if pdf_path:
            print(f"Статья {result.entry_id} успешно скачана и сохранена в {pdf_path}")
            # Добавление arXiv ID в список скачанных
            downloaded_arxiv_ids.append(result.entry_id.split("/")[-1])
    # Сохранение списка скачанных arXiv ID в файл JSON
    with open(saved_list_path, "w") as f:
        json.dump(downloaded_arxiv_ids, f)

In [105]:
# Пример использования функции
download_from_arxiv("RAG", max_results=2)

Статья http://arxiv.org/abs/2404.15488v1 уже скачана и пропущена.
Статья http://arxiv.org/abs/2404.14043v1 уже скачана и пропущена.


In [106]:
def get_pdf_files(directory):
    pdf_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

In [107]:
# Пример использования функции
directory = "assets/arxiv/"
pdf_files = get_pdf_files(directory)
for pdf_file in pdf_files:
    print(pdf_file)

assets/arxiv/2404.14043v1.LLMs_Know_What_They_Need__Leveraging_a_Missing_Information_Guided_Framework_to_Empower_Retrieval_Augmented_Generation.pdf
assets/arxiv/2404.15488v1.IryoNLP_at_MEDIQA_CORR_2024__Tackling_the_Medical_Error_Detection___Correction_Task_On_the_Shoulders_of_Medical_Agents.pdf


In [111]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("assets/arxiv/")
# loader = UnstructuredPDFLoader(pdf_files)
# loader = PyPDFLoader(pdf_files)
docs = loader.load()

split_docs = CharacterTextSplitter(chunk_size=5000, chunk_overlap=1000).split_documents(
    docs
)
print(f"Parts count: {len(split_docs)}")

Parts count: 32


In [113]:
from langchain.chains.summarize import load_summarize_chain

giga = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)
chain = load_summarize_chain(giga, chain_type="map_reduce")
res = chain.run(split_docs)

print("\n\n===")
print(res)

/workspace/Syrenny/GigaHack/Syrenny/giga_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




===
This paper presents the MIGRES framework for improving Retrieval-Augmented Generation (RAG) in Large Language Models (LLMs) by leveraging missing information to generate targeted queries and filter out irrelevant content. The framework also includes an information extraction capability of LLMs. Extensive experiments demonstrate the superiority of the proposed method. Additionally, the MedReAct'N'MedReFlex framework is presented as a multi-agent system for medical error detection and correction in clinical notes. The system incorporates four specialized medical agents and five evaluators for comprehensive feedback. The framework optimizes its semantic search engine and achieves the ninth rank in a competition with an aggregation score of 0.581.
